In [1]:
import pandas as pd
import numpy as np
import sklearn

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

### 讀入資料並判斷缺數值、非數字

In [59]:
org_data = pd.read_csv('HW3_hr-analytics.csv')

 ### 前處理

In [60]:
OH_data = org_data.copy()

lin_i = [0,1,3]# 非離散型資料(col)

for i,col in enumerate(OH_data.columns):
    if i ==0:
        std = MinMaxScaler()
        matrix = np.array(OH_data[col])[:, np.newaxis]
        std.fit(matrix)
        matrix = std.transform(matrix)

    elif i  in  lin_i:
        OH_temp = np.array(OH_data[col])[:, np.newaxis]
        std = MinMaxScaler()
        std.fit(OH_temp)
        OH_temp = std.transform(OH_temp)
        matrix = np.hstack((matrix,OH_temp))
    #切割目標
    elif col == 'left':
        OH_temp = np.array(OH_data[col])
        target = OH_temp[:,np.newaxis]
    else:
        #OnehotEncoding       
        OH = OneHotEncoder()
        OH_temp = np.array(OH_data[col])
        OH_temp = OH_temp[:,np.newaxis]
        OH.fit(OH_temp)
        OH_temp = OH.transform(OH_temp).toarray()
        matrix = np.hstack((matrix,OH_temp))


### 切割資料

In [61]:
train_data, test_data ,train_target , test_target = train_test_split(matrix,target,test_size=0.3)  

 ### 決策樹模型模型

In [62]:
dec = DecisionTreeClassifier()
dec.fit(train_data, train_target.ravel())
test_pred = dec.predict(test_data)

corr = np.sum(np.array(test_pred)[:, np.newaxis] == test_target)
length = len(test_data)
score = dec.score(test_data, test_target)*100

#評分
print(f'accuracy : {corr}/{length}  {score:.2f}%')


accuracy : 4398/4500  97.73%


### 神經網絡模型

In [63]:
mlp = MLPClassifier(max_iter=3000)
mlp.fit(train_data,train_target.ravel())
test_pred = mlp.predict(test_data)

corr = np.sum(np.array(test_pred)[:, np.newaxis] == test_target)
length = len(test_data)
score = dec.score(test_data, test_target)*100

#評分
print(f'accuracy : {corr}/{length}  {score:.2f}%')


accuracy : 4346/4500  97.73%


### 邏輯迴歸模型比較

In [52]:
log = LogisticRegression(max_iter=1000)
log.fit(train_data, train_target.ravel())
test_pred = log.predict(test_data)

corr = np.sum(np.array(test_pred)[:, np.newaxis] == test_target)
length = len(test_data)
score = log.score(test_data, test_target)*100

#評分
print(f'accuracy : {corr}/{length}  {score:.2f}%')


accuracy : 4048/4500  89.96%


 #### 發現邏輯迴歸之準確率較低，因為邏輯迴歸是以一線性函數線進行分類判斷，決策樹則以各節點對應目標值的最優解判定，故在分類上決策樹會比羅輯迴歸更準確